In [1]:
import pandas as pd
import numpy as np
import cvxpy as cvx

In [2]:
equity_list = ['DELL', 'IBM', 'MSFT', 'ORCL']
alpha_list = [-0.05, 0.07, 0.04, -0.02]
# alpha_list = [-0.006, 0.0105, 0.0233, -0.0077] # implied alpha
alpha_series = pd.Series(index=equity_list, data=alpha_list)
cov_list = [
    [0.02, 0.0001, 0.002, -0.0001],
    [0.0001, 0.04, 0.0003, 0.0009],
    [0.002, 0.0003, 0.05, 0.0001],
    [-0.0001, 0.0009, 0.0001, 0.02]
]
cov_df = pd.DataFrame(index=equity_list, columns=equity_list, data=cov_list)

In [3]:
print(alpha_series)
print('\n')
print(cov_df)

DELL   -0.05
IBM     0.07
MSFT    0.04
ORCL   -0.02
dtype: float64


        DELL     IBM    MSFT    ORCL
DELL  0.0200  0.0001  0.0020 -0.0001
IBM   0.0001  0.0400  0.0003  0.0009
MSFT  0.0020  0.0003  0.0500  0.0001
ORCL -0.0001  0.0009  0.0001  0.0200


In [4]:
reference_size = 1.
risk_avn = 1. / reference_size

# 约束条件

In [5]:
ub = pd.Series()
lb = pd.Series()
group_b = {}

In [6]:
pre_holding = pd.Series(index=equity_list, data=0.)
pre_holding['cash'] = reference_size
current_price = pd.Series(index=equity_list+['cash'], data=1.)

# 输入

In [7]:
# alpha和cov扩展ｃａｓｈ
alpha_series['cash'] = 0.
cov_df = cov_df.reindex(equity_list + ['cash']).fillna(0.)
cov_df['cash'] = 0.
print(alpha_series)
print('\n')
print(cov_df)
trading_day = '20100101'
cov_panel = pd.Panel(
    data={trading_day: cov_df})
annual_rt_df = pd.DataFrame(
    index=[trading_day],
    data=[alpha_series])
# benchmark
benchmark_weight = pd.Series(index=equity_list+['cash'], data=[0.3, 0.23, 0.12, 0.35, 0.])

DELL   -0.05
IBM     0.07
MSFT    0.04
ORCL   -0.02
cash    0.00
dtype: float64


        DELL     IBM    MSFT    ORCL  cash
DELL  0.0200  0.0001  0.0020 -0.0001   0.0
IBM   0.0001  0.0400  0.0003  0.0009   0.0
MSFT  0.0020  0.0003  0.0500  0.0001   0.0
ORCL -0.0001  0.0009  0.0001  0.0200   0.0
cash  0.0000  0.0000  0.0000  0.0000   0.0


/home/casey/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3325: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  exec(code_obj, self.user_global_ns, self.user_ns)


# CVXPY配置

In [8]:
from solar.cvxportfolio.constraints import LongCash, LongOnly
from solar.cvxportfolio.costs import TcostModel
from solar.cvxportfolio.returns import ReturnsForecast
from solar.solver.components.constraints import AssetWeightBound, TurnoverLimit, GroupWeightBound
from solar.solver.components.policies import MaxUtilityPolicy
from solar.solver.components.risks import AssetSigma
from solar.math.common_function import adjust_holding_precision
solver_param = {}
solver_param.update({
    cvx.ECOS: {
        'abstol': 1.0e-09,
        'reltol': 1.0e-09,
        'max_iters': 150
    }
})

In [9]:
class MaxUtilityFunctionSolver:
    def __init__(self, param):
        self.asset_list = param['asset_list']
        self.asset_cov = param['asset_cov']
        self.asset_rt = param['asset_rt']
        self.risk_aversion = param['risk_aversion']
        self.total_weight = param['total_weight']
        self.target_turnover = param['target_turnover']
        self.fee = param['fee']
        self.lower_boundary = param['lb']
        self.upper_boundary = param['ub']
        self.group_boundary = param['group_b']
        self.trading_day = param['trading_day']
        self.pre_holding = param['pre_holding']
        self.current_price = param['current_price']
        self.solver = param['solver']
        self.weight_precision = param['weight_precision']
        self.benchmark_weight_series = param['benchmark_weight_series']
        self.is_long_only = param['is_long_only']
        self.is_long_cash = param['is_long_cash']
        self.lower_boundary = self.lower_boundary.reindex(self.asset_list).fillna(-np.inf)
        self.upper_boundary = self.upper_boundary.reindex(self.asset_list).fillna(np.inf)

    def _analysis_dual_variable(self, prob, variable):
        inside_constraint_list = prob.constraints
        dual_constraint_dict = {}
        for constraint in inside_constraint_list:
            dual_value = constraint.dual_value  # lambda
            jac_matrix = constraint.expr.grad[variable].toarray()
            dual_constraint_dict[constraint] = (dual_value, jac_matrix)
        return dual_constraint_dict

    def _constraint_attribution(self, prob, dual_constraint_dict):
        inside_constraint_list = prob.constraints
        alpha_decomp_dict = {}
        holding_decomp_dict = {}
        expected_return_decomp_dict = {}
        cov_arr = self.asset_cov.values[0]
        cov_arr = cov_arr[:len(self.asset_list), :len(self.asset_list)]
        inv_cov_arr = np.linalg.inv(cov_arr)
        inv_cov_df = pd.DataFrame(index=self.asset_list, columns=self.asset_list, data=inv_cov_arr)
        inv_cov_df = inv_cov_df.reindex(self.asset_list + ['cash']).fillna(0.)
        inv_cov_df['cash'] = 0.
#         inv_cov_df.loc['cash', 'cash'] = 1.
#         self.asset_rt.loc[self.trading_day]['cash'] = 1.
        for constraint in inside_constraint_list:
            dual_value, jac_matrix = dual_constraint_dict[constraint]
            # alpha decomposition
            cur_alpha_decomp_mat = (-1 * jac_matrix * dual_value).T
            alpha_decomp_dict[constraint.name()] = cur_alpha_decomp_mat
            # holding decomposition
            cur_holding_decomp_mat = cur_alpha_decomp_mat.dot(inv_cov_df)
            holding_decomp_dict[constraint.name()] = cur_holding_decomp_mat
            # expected return decomposition
            expected_return_decomp_dict[constraint.name()] = cur_holding_decomp_mat.dot(self.asset_rt.loc[self.trading_day])
        result = {
            'alpha_decomp_dict': alpha_decomp_dict,
            'holding_decomp_dict': holding_decomp_dict,
            'expected_return_decomp_dict': expected_return_decomp_dict
        }
        return result


    def solve(self):
        transaction_cost_mdl = TcostModel(self.fee) if self.fee is not None else None
        cost_models = []
        if transaction_cost_mdl is not None:
            cost_models.append(transaction_cost_mdl)

        asset_cov_mdl = AssetSigma(self.asset_cov, w_bench=self.benchmark_weight_series)
        return_mdl = ReturnsForecast(self.asset_rt)
        weight_bound_list = [x for x in zip(self.lower_boundary, self.upper_boundary)]
        if self.total_weight:
            # weight_bound_list += [1 - self.total_weight]
            self.group_boundary.append(
                {'index': [i for i in range(len(self.asset_list))], 'cb': self.total_weight}
            )
        wt_cons = AssetWeightBound(weight_bound_list)

        group_wt_cons = GroupWeightBound(self.group_boundary)
        constraints = []
        if self.is_long_only:
            constraints += [LongOnly()]
        if self.is_long_cash:
            constraints += [LongCash()]
        constraints = [wt_cons, group_wt_cons]

        if self.target_turnover is not None:
            turnover_cons = TurnoverLimit(self.target_turnover)
            constraints.append(turnover_cons)

        policy = MaxUtilityPolicy(
            return_mdl,
            asset_cov_mdl,
            self.risk_aversion,
            costs=cost_models,
            constraints=constraints,
            solver=self.solver,
            solver_opts=solver_param[self.solver])
        prob, trade_value = policy.get_trades(self.pre_holding, self.trading_day)
        dual_constraint_dict = self._analysis_dual_variable(prob, policy.variable)
        constraint_attr_result = \
            self._constraint_attribution(prob, dual_constraint_dict)
        trade_volume = trade_value.drop('cash') / self.current_price
        trade_volume.fillna(0., inplace=True)
        after_trade_value = self.pre_holding + trade_value
        after_trade_ratio = after_trade_value / after_trade_value.sum()
        after_trade_ratio = adjust_holding_precision(after_trade_ratio, self.weight_precision)
        transaction_cost_sum = 0.
        for cost in policy.costs:
            if isinstance(cost, TcostModel):
                transaction_cost_sum += cost.value_expr(0., 0., trade_value)
        trade_result = {
            'trade_value': trade_value,
            'trade_volume': trade_volume,
            'solution_status': prob.status,
            'transaction_cost_sum': transaction_cost_sum,
            'weight': after_trade_ratio,
            'holding': after_trade_value,
            'constraint_attr_result': constraint_attr_result
        }
        return trade_result


In [10]:
free_param = {'asset_list': equity_list,
         'asset_cov': cov_panel,
         'asset_rt': annual_rt_df,
         'risk_aversion': risk_avn,
         'total_weight': None,
         'target_turnover': None,
         'fee': None,
         'lb': lb,
         'ub': ub,
         'group_b': group_b,
         'trading_day': trading_day,
         'pre_holding': pre_holding,
         'current_price': current_price,
         'solver': cvx.ECOS,
         'weight_precision': 0.00000001,
         'benchmark_weight_series': benchmark_weight,
         'is_long_only': False,
         'is_long_cash': False
        }

In [11]:
model = MaxUtilityFunctionSolver(free_param)
free_result = model.solve()

/home/casey/anaconda3/lib/python3.7/site-packages/pandas/core/dtypes/missing.py:99: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  return _isna(obj)


In [12]:
free_result['holding']

DELL   -2.303920
IBM     2.004482
MSFT    1.015705
ORCL   -0.747341
cash    1.031074
dtype: float64

In [13]:
# expected return
free_result['holding'].dot(alpha_series)

0.3110847699676577

In [14]:
equity_list

['DELL', 'IBM', 'MSFT', 'ORCL']

In [15]:
ub = pd.Series(index=equity_list, data=[np.inf, 0.5, np.inf, np.inf])
lb = pd.Series(index=equity_list, data=-0.05)
total_weight = 1.
group_b = []
constraint_param = {'asset_list': equity_list,
         'asset_cov': cov_panel,
         'asset_rt': annual_rt_df,
         'risk_aversion': risk_avn,
         'total_weight': total_weight,
         'target_turnover': None,
         'fee': None,
         'lb': lb,
         'ub': ub,
         'group_b': group_b,
         'trading_day': trading_day,
         'pre_holding': pre_holding,
         'current_price': current_price,
         'solver': cvx.ECOS,
         'weight_precision': 0.00001,
         'benchmark_weight_series': benchmark_weight,
         'is_long_only': False,
         'is_long_cash': False
        }

In [16]:
model = MaxUtilityFunctionSolver(constraint_param)
constraint_result = model.solve()

In [17]:
constraint_result['holding']

DELL   -5.000000e-02
IBM     5.000000e-01
MSFT    6.000000e-01
ORCL   -5.000000e-02
cash    5.984102e-14
dtype: float64

In [18]:
# expected return
constraint_result['holding'].dot(alpha_series)

0.06249999997116984

In [19]:
holding_decomp_arr = None
for key, value in constraint_result['constraint_attr_result']['holding_decomp_dict'].items():
    if holding_decomp_arr is None:
        holding_decomp_arr = value
    else:
        holding_decomp_arr = np.concatenate([holding_decomp_arr, value])
print(holding_decomp_arr.sum(axis=0) + free_result['holding'].values)

[-0.04998528  0.49999262  0.59999103 -0.0499805   1.03107427]


# alpha decomposition

个股权重设置了下限为$-5\%$，IBM的上限为$50\%$，总权重为$1$，共6个约束条件，其中有5个为不等式约束，1个为等式约束。
原目标函数为
$$
\max \alpha^T x - \frac{1}{2} (x-b)^T Q (x-b)
$$
将约束条件带入原目标函数中，利用拉格朗日乘子法，得到拉格朗日函数
$$
L(x, \lambda_{1-6})= \alpha^T x - \frac{1}{2} (x-b)^T Q (x-b) - \lambda_1(\sum_{i=2}^5 x_i - 1) - \sum_{i=2}^5(\lambda_i(-0.05-x_i)) - \lambda_6(x_2-0.5)
$$
其中，乘子$\lambda_1$对应的是等式约束，$\lambda_{2-6}$对应的是不等式约束。

拉格朗日函数$L$对变量$x$求导，可得
$$
\frac{\partial L}{\partial x} = \alpha - Q^T(x-b) - \lambda_1 I + [\lambda_2,\lambda_3,\lambda_4,\lambda_5]^T - [0,\lambda_6,0,0]^T
$$
令偏导数等于0，整理两边可得
$$
\alpha - Q^T(x^*-b) = 
\left[
  \begin{matrix}
   \lambda_1^*-\lambda_2^*\\
   \lambda_1^*-\lambda_3^*-\lambda_6^*\\
   \lambda_1^*-\lambda_4^*\\
   \lambda_1^*-\lambda_5^*\\
  \end{matrix}
\right]
$$


不等式约束需要满足根据KKT条件，这里只列出相关的条件
$$
\lambda_2^*(-0.05-x_1^*)=0 \\
\lambda_3^*(-0.05-x_2^*)=0 \\
\lambda_4^*(-0.05-x_3^*)=0 \\
\lambda_5^*(-0.05-x_4^*)=0 \\
\lambda_6^*(x_2^*-0.5)=0 \\
$$

通过求解的结果可以看到，DELL、IBM、ORCL达到了约束条件的边界，它们分别的权重是

$w^*_{\text{DELL}}=-0.05,w^*_{\text{IBM}}=0.5,w^*_{\text{ORCL}}=-0.05$。

因此，这些约束对应的乘子$\lambda \neq 0$，对应的乘子有$\lambda_2^*,\lambda_6^*,\lambda_5^*$。因此其他不等式约束对应的乘子为$0$。这里有$\lambda_3^*=0,\lambda_4^*=0$。

联系偏导数得到的关于$\lambda$的等式，可以求的所有的$\lambda^*$。

In [20]:
# implied alpha
def implied_alpha(cov_df, optimal_weight, wb_bench):
    return cov_df.dot(optimal_weight - wb_bench)/reference_size
im_alpha = implied_alpha(cov_df, constraint_result['holding'], benchmark_weight)
im_alpha

DELL   -0.005973
IBM     0.010549
MSFT    0.023341
ORCL   -0.007674
cash    0.000000
dtype: float64

In [23]:
lm = alpha_series - im_alpha
lm

DELL   -0.044027
IBM     0.059451
MSFT    0.016659
ORCL   -0.012326
cash    0.000000
dtype: float64

In [24]:
# budget alpha
lm4 = 0
lm3 = 0
lm1 = lm.iloc[2] + lm4
lm2 = lm1 - lm.iloc[0]
lm5 = lm1 - lm.iloc[3]
lm6 = lm1 - lm.iloc[1] - lm3

In [25]:
pd.Series(index=['lm1','lm２','lm３','lm４','lm５','lm６'], data=[lm1,lm2,lm3,lm4,lm5,lm6])

lm1    0.016659
lm２    0.060686
lm３    0.000000
lm４    0.000000
lm５    0.028985
lm６   -0.042792
dtype: float64

$\lambda$和对应的约束条件相乘后，就是该约束条件对应的alpha分解的部分。以等式约束$\sum_i x_i=1$为例，alpha分解到这个部分是可以从偏$L$关于$x$的偏导数中看出，为$-\lambda_1 I=[-\lambda_1,-\lambda_1,-\lambda_1,-\lambda_1]=[-0.0167,-0.0167,-0.0167,-0.0167]$。

DELL下限被分解到的alpha为$[\lambda_2,0,0,0]=[0.0607,0,0,0]$

IBM下限被分解到的alpha为$[0,\lambda_3,0,0]=[0,0,0,0]$

MSFT下限被分解到的alpha为$[0,0,\lambda_4,0]=[0,0,0,0]$

ORCL下限被分解到的alpha为$[0,0,0,\lambda_5]=[0,0,0,0.0290]$

IBM上限被分解到的alpha为$[0,\lambda_6,0,0]=[0,-0.0428,0,0]$

## holding decomposition

In [26]:
# inverse cov_df
cov_list = [
    [0.02, 0.0001, 0.002, -0.0001],
    [0.0001, 0.04, 0.0003, 0.0009],
    [0.002, 0.0003, 0.05, 0.0001],
    [-0.0001, 0.0009, 0.0001, 0.02]
]
cov_df = pd.DataFrame(index=equity_list, columns=equity_list, data=cov_list)
inv_cov_arr = np.linalg.inv(cov_df)
inv_cov_df = pd.DataFrame(index=equity_list, columns=equity_list, data=inv_cov_arr)
# inv_cov_df = inv_cov_df.reindex(equity_list + ['cash']).fillna(0.)
# inv_cov_df['cash'] = 0.

In [28]:
def portfolio_decomp(inv_cov_df, alpha_series):
    active_free_holding = inv_cov_df.dot(alpha_series[equity_list])
    active_budget_holding = -inv_cov_df.sum(axis=1)*lm1
    lower_bound_holding = -inv_cov_df * pd.Series(index=equity_list, data=[lm2,lm3,lm4,lm5])
    return active_free_holding, active_budget_holding, lower_bound_holding
active_free_holding, active_budget_holding, lower_bound_holding = portfolio_decomp(inv_cov_df, alpha_series)
lower_bound_holding.T

,DELL,IBM,MSFT,ORCL
DELL,-3.046602,0.007066,0.121854,-0.016160
IBM,0.000000,-0.000000,0.000000,0.000000
MSFT,0.000000,0.000000,-0.000000,0.000000
ORCL,-0.007719,0.032639,0.003014,-1.450772


In [223]:
inv_cov_df.dot(np.diag(pd.Series(index=equity_list, data=[lm2,lm3,lm4,lm5])))

,0,1,2,3
DELL,3.046602,0.0,0.0,0.007719
IBM,-0.007066,0.0,0.0,-0.032639
MSFT,-0.121854,0.0,0.0,-0.003014
ORCL,0.016160,0.0,0.0,1.450772


In [224]:
inv_cov_df.dot(pd.Series(index=equity_list, data=[lm2,0,0,0]))

DELL    3.046602
IBM    -0.007066
MSFT   -0.121854
ORCL    0.016160
dtype: float64

In [225]:
inv_cov_df.dot(pd.Series(index=equity_list, data=[0,lm6,0,0]))

DELL    0.004983
IBM    -1.070943
MSFT    0.006130
ORCL    0.048187
dtype: float64

## 实际的例子

In [29]:
equity_list = ['TCL集团','潍柴动力','云南白药','泸州老窖','天茂集团','格力电器','恒逸石化','京东方A','中航飞机','长江证券']
cov_arr = np.array([[0.0892,0.0324,0.0217,0.0299,0.0316,0.0295,0.0281,0.0413,0.0267,0.0379],
[0.0324,0.0861,0.0230,0.0335,0.0296,0.0313,0.0275,0.0347,0.0222,0.0354],
[0.0217,0.0230,0.0442,0.0263,0.0184,0.0232,0.0192,0.0239,0.0154,0.0226],
[0.0299,0.0335,0.0263,0.0934,0.0247,0.0340,0.0259,0.0327,0.0187,0.0320],
[0.0316,0.0296,0.0184,0.0247,0.0924,0.0257,0.0268,0.0326,0.0241,0.0498],
[0.0295,0.0313,0.0232,0.0340,0.0257,0.0770,0.0242,0.0326,0.0196,0.0313],
[0.0281,0.0275,0.0192,0.0259,0.0268,0.0242,0.0748,0.0293,0.0206,0.0306],
[0.0413,0.0347,0.0239,0.0327,0.0326,0.0326,0.0293,0.0927,0.0279,0.0400],
[0.0267,0.0222,0.0154,0.0187,0.0241,0.0196,0.0206,0.0279,0.0427,0.0278],
[0.0379,0.0354,0.0226,0.0320,0.0498,0.0313,0.0306,0.0400,0.0278,0.0881]])
cov_df = pd.DataFrame(index=equity_list, columns=equity_list, data=cov_arr)

alpha_list = [-0.9793 ,0.7198 ,1.9708 ,1.7054 ,-0.7421 ,1.2155 ,-0.2691 ,-1.7659 ,-0.8677 ,-0.6070]
alpha_series = pd.Series(index=equity_list, data=alpha_list)/100.

# benchmark
benchmark_weight_list = [0.0584 ,0.0933 ,0.0615 ,0.0933 ,0.0206 ,0.4136 ,0.0213 ,0.1542 ,0.0353 ,0.0487]
benchmark_weight = pd.Series(index=equity_list, data=benchmark_weight_list)
benchmark_weight['cash'] = 0.

alpha_series['cash'] = 0.
cov_df = cov_df.reindex(equity_list + ['cash']).fillna(0.)
cov_df['cash'] = 0.
print(alpha_series)
print('\n')
print(cov_df)
trading_day = '20100101'
cov_panel = pd.Panel(
    data={trading_day: cov_df})
annual_rt_df = pd.DataFrame(
    index=[trading_day],
    data=[alpha_series])

pre_holding = pd.Series(index=equity_list, data=0.)
pre_holding['cash'] = 1.
print(pre_holding)
current_price = pd.Series(index=equity_list, data=1.)
current_price['cash'] = 1.

TCL集团   -0.009793
潍柴动力     0.007198
云南白药     0.019708
泸州老窖     0.017054
天茂集团    -0.007421
格力电器     0.012155
恒逸石化    -0.002691
京东方A    -0.017659
中航飞机    -0.008677
长江证券    -0.006070
cash     0.000000
dtype: float64


        TCL集团    潍柴动力    云南白药    泸州老窖    天茂集团    格力电器    恒逸石化    京东方A    中航飞机  \
TCL集团  0.0892  0.0324  0.0217  0.0299  0.0316  0.0295  0.0281  0.0413  0.0267   
潍柴动力   0.0324  0.0861  0.0230  0.0335  0.0296  0.0313  0.0275  0.0347  0.0222   
云南白药   0.0217  0.0230  0.0442  0.0263  0.0184  0.0232  0.0192  0.0239  0.0154   
泸州老窖   0.0299  0.0335  0.0263  0.0934  0.0247  0.0340  0.0259  0.0327  0.0187   
天茂集团   0.0316  0.0296  0.0184  0.0247  0.0924  0.0257  0.0268  0.0326  0.0241   
格力电器   0.0295  0.0313  0.0232  0.0340  0.0257  0.0770  0.0242  0.0326  0.0196   
恒逸石化   0.0281  0.0275  0.0192  0.0259  0.0268  0.0242  0.0748  0.0293  0.0206   
京东方A   0.0413  0.0347  0.0239  0.0327  0.0326  0.0326  0.0293  0.0927  0.0279   
中航飞机   0.0267  0.0222  0.0154  0.0187  0.0241  0.0196  0

/home/casey/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3325: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  exec(code_obj, self.user_global_ns, self.user_ns)


In [30]:
ub = pd.Series(index=equity_list, data=np.inf)
lb = pd.Series(index=equity_list, data=-np.inf)
total_weight = None
group_b = []
free_param = {
    'asset_list': equity_list,
     'asset_cov': cov_panel,
     'asset_rt': annual_rt_df,
     'risk_aversion': risk_avn,
     'total_weight': total_weight,
     'target_turnover': None,
     'fee': None,
     'lb': lb,
     'ub': ub,
     'group_b': group_b,
     'trading_day': trading_day,
     'pre_holding': pre_holding,
     'current_price': current_price,
     'solver': cvx.ECOS,
     'weight_precision': 0.00001,
     'benchmark_weight_series': benchmark_weight,
     'is_long_only': False,
     'is_long_cash': False
}

In [31]:
model = MaxUtilityFunctionSolver(free_param)
free_result = model.solve()

In [32]:
print(free_result['holding'])

TCL集团   -0.063867
潍柴动力     0.185847
云南白药     0.662650
泸州老窖     0.259453
天茂集团    -0.038668
格力电器     0.578599
恒逸石化    -0.036902
京东方A    -0.151128
中航飞机    -0.205965
长江证券    -0.009492
cash    -0.180526
dtype: float64


In [33]:
# expected return
free_result['holding'].dot(alpha_series)

0.031380072218976054

In [34]:
ub = pd.Series(index=equity_list, data=np.inf)
lb = pd.Series(index=equity_list, data=0.)
# lb['长江证券'] = -np.inf
ub['云南白药'] = 0.2
total_weight = 1.
group_b = []
constraint_param = {
    'asset_list': equity_list,
     'asset_cov': cov_panel,
     'asset_rt': annual_rt_df,
     'risk_aversion': risk_avn,
     'total_weight': total_weight,
     'target_turnover': None,
     'fee': None,
     'lb': lb,
     'ub': ub,
     'group_b': group_b,
     'trading_day': trading_day,
     'pre_holding': pre_holding,
     'current_price': current_price,
     'solver': cvx.ECOS,
     'weight_precision': 0.00001,
     'benchmark_weight_series': benchmark_weight,
     'is_long_only': False,
     'is_long_cash': False
}

In [35]:
model = MaxUtilityFunctionSolver(constraint_param)
constraint_result = model.solve()

In [36]:
print(constraint_result['holding'])

TCL集团    1.525256e-11
潍柴动力     8.518756e-02
云南白药     2.000000e-01
泸州老窖     2.249840e-01
天茂集团     1.626033e-11
格力电器     4.898284e-01
恒逸石化     2.097170e-11
京东方A     1.173659e-11
中航飞机     1.349161e-11
长江证券     1.985121e-11
cash    -4.019007e-14
dtype: float64


In [37]:
# expected return
constraint_result['holding'].dot(alpha_series)

0.01434552185931314

In [38]:
holding_decomp_arr = None
for key, value in constraint_result['constraint_attr_result']['holding_decomp_dict'].items():
    if holding_decomp_arr is None:
        holding_decomp_arr = value
    else:
        holding_decomp_arr = np.concatenate([holding_decomp_arr, value])
print(holding_decomp_arr.sum(axis=0) + free_result['holding'].values)
# print(holding_decomp_arr)

[-3.81226443e-07  8.51876806e-02  2.00002818e-01  2.24983834e-01
  1.35610258e-07  4.89828500e-01 -2.51099889e-07 -1.65405389e-06
  1.28133644e-07 -4.81197910e-07 -1.80525643e-01]


In [39]:
alpha_decomp_arr = None
for key, value in constraint_result['constraint_attr_result']['alpha_decomp_dict'].items():
    if alpha_decomp_arr is None:
        alpha_decomp_arr = value
    else:
        alpha_decomp_arr = np.concatenate([alpha_decomp_arr, value])
print(alpha_decomp_arr.sum(axis=0) + alpha_series.values)


[-6.68683398e-03 -1.66169127e-03  3.78205161e-03  8.19430873e-03
 -5.10272504e-03  3.29530873e-03 -2.83040420e-03 -1.11151391e-02
 -3.74852207e-03 -5.88813262e-03  2.95289316e-20]


In [40]:
implied_alpha(cov_df, constraint_result['holding'], benchmark_weight)

TCL集团   -0.006687
潍柴动力    -0.001662
云南白药     0.003782
泸州老窖     0.008194
天茂集团    -0.005103
格力电器     0.003295
恒逸石化    -0.002830
京东方A    -0.011115
中航飞机    -0.003748
长江证券    -0.005888
cash     0.000000
dtype: float64

In [41]:
# expected return
expected_return_decomp_arr = None
for key, value in constraint_result['constraint_attr_result']['expected_return_decomp_dict'].items():
    if expected_return_decomp_arr is None:
        expected_return_decomp_arr = value
    else:
        expected_return_decomp_arr = np.concatenate([expected_return_decomp_arr, value])
# print(alpha_decomp_arr.sum(axis=0) + alpha_series.values)

In [184]:
# constraint_result['constraint_attr_result']['expected_return_decomp_dict'].keys()

In [185]:
# constraint_result['constraint_attr_result']['expected_return_decomp_dict']['w_plus[2] <= [0.2]']

In [186]:
# expected_return_decomp_arr

In [47]:
constraint_result['constraint_attr_result']['holding_decomp_dict'].keys()

dict_keys(['[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] <= w_plus[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]', 'w_plus[2] <= [0.2]', 'Sum(w_plus[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], None, False) == 1.0', 'Sum(w_plus + -[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.], None, False) == 0.0'])

In [53]:
pd.DataFrame(constraint_result['constraint_attr_result']['holding_decomp_dict']['Sum(w_plus[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], None, False) == 1.0'])

,0,1,2,3,4,5,6,7,8,9,10
0,-0.006668,-0.012786,-0.110952,-0.013285,-0.022768,-0.025249,-0.03507,0.004987,-0.11934,0.010145,0.0


In [329]:
constraint_result['constraint_attr_result']['alpha_decomp_dict'].keys()

dict_keys(['[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] <= w_plus[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]', 'w_plus[2] <= [0.2]', 'Sum(w_plus[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], None, False) == 1.0', 'Sum(w_plus + -[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.], None, False) == 0.0'])

In [332]:
pd.DataFrame(constraint_result['constraint_attr_result']['alpha_decomp_dict']['w_plus[2] <= [0.2]'])

,0,1,2,3,4,5,6,7,8,9,10
0,-0.0,-0.0,-0.007066,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0


In [37]:
inv_cov_arr = np.linalg.inv(cov_df.loc[equity_list, equity_list])
inv_cov_df = pd.DataFrame(index=equity_list, columns=equity_list, data=inv_cov_arr)

In [38]:
inv_cov_df.dot(alpha_series[equity_list]) + benchmark_weight[equity_list]

TCL集团   -0.063867
潍柴动力     0.185847
云南白药     0.662650
泸州老窖     0.259453
天茂集团    -0.038668
格力电器     0.578598
恒逸石化    -0.036902
京东方A    -0.151128
中航飞机    -0.205965
长江证券    -0.009492
dtype: float64